In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np;
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)


In [5]:
# tf.reset_default_graph()
# sess = tf.InteractiveSession()
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)#, reuse=True)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
#     한번 정의 하면 다시 안만들어두 됨, one_cell scope 내에 이미 생성
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
# basic rnn 에서는 output[-1] = _state

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    pp.pprint(_states.eval())    

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[-0.4475658,  0.4782141]]], dtype=float32)
array([[-0.4475658,  0.4782141]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [91]:
# tf.reset_default_graph()
# sess = tf.InteractiveSession()
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.4117932 , -0.6803914 ],
        [-0.41059777, -0.22151004],
        [ 0.24800533, -0.5206909 ],
        [ 0.38022932, -0.7784485 ],
        [ 0.36700892, -0.5515955 ]]], dtype=float32)
array([[ 0.36700892, -0.5515955 ]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [40]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
#     outputs, _states = tf.nn.dynamic_rnn(
#         cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    
    pp.pprint(outputs.eval())
    pp.pprint(sess.run(_states))

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.10182311, -0.15088622],
        [-0.13941534, -0.2025134 ],
        [-0.07251859, -0.15943602],
        [-0.07341124, -0.19699812],
        [-0.21873139, -0.14464548]],

       [[-0.05208891, -0.14652072],
        [-0.17189854, -0.07478711],
        [-0.08997323, -0.14334752],
        [-0.08414454, -0.18234287],
        [-0.07684471, -0.20861493]],

       [[-0.0330142 , -0.09741466],
        [-0.04702163, -0.1563304 ],
        [-0.08649664, -0.271051  ],
        [-0.07334071, -0.2483477 ],
        [-0.04573505, -0.1745006 ]]], dtype=float32)
LSTMStateTuple(c=array([[-0.

In [41]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.07617827, -0.04428805],
        [ 0.03876524,  0.08833982],
        [ 0.09856242,  0.2464367 ],
        [ 0.14265995,  0.34341532],
        [ 0.14838411,  0.22156611]],

       [[ 0.09071095,  0.11328734],
        [ 0.09160928,  0.03934482],
        [ 0.13803647,  0.17760944],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.07549731,  0.14988899],
        [ 0.12693031,  0.27127272],
        [ 0.20409304,  0.25830266],
        [ 0.24315977,  0.28867972],
        [ 0.        ,  0.        ]]], dtype=float32)


In [42]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.03993475, -0.11892224],
        [-0.12191749, -0.0420336 ],
        [-0.03652245, -0.12939273],
        [-0.00636312, -0.16936366],
        [ 0.08314894, -0.13605908]],

       [[-0.13643199, -0.00222773],
        [ 0.0299538 , -0.0763106 ],
        [ 0.03364702, -0.17137234],
        [ 0.03755234, -0.19780532],
        [ 0.03830506, -0.2171274 ]],

       [[ 0.0224665 , -0.07741912],
        [ 0.03418965, -0.13172509],
        [-0.12037027, -0.04724158],
        [-0.19744764, -0.03196891],
        [-0.07549693, -0.11937114]]], dtype=float32)


In [97]:
np.arange(50).reshape([-1,5])

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [13]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [43]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32) # [batch_size, state_size] filled with zeros.
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[-0.09132701, -0.07079794,  0.04599177,  0.01998146,
         -0.01158326],
        [-0.06252545, -0.04564673,  0.10397587, -0.00046265,
          0.03994242],
        [ 0.01154837, -0.00860379,  0.12329465,  0.0591893 ,
         -0.03139519],
        [ 0.09271022,  0.0431072 ,  0.14971118,  0.09715822,
         -0.06826583],
        [ 0.03395345,  0.03721558,  0.17606905,  0.07806135,
         -0.00537324]],

       [[ 0.02376806, -0.00639667,  0.06301279, -0.03489038,
          0.06220614],
        [-0.02859783, -0.02447187,  0.11615314, -0.03051697,
          0.06576603],
        [ 0.04705526,  0.01413366,  0.14653279,  0.0423134 ,
         -0.00669572],
        [ 0.11618993,  0.05798095,  0.16416143,  0.08566584,
         -0.05308986],
        [ 0.17427595,  0.09740557,  0.17279524,  0.11246333,
         -0.08261189]],

       [[ 0.08939864,  0.03137995,  0.07131325,  0.04722066,
         -0.03754198],
        [ 0.1593059 ,  0.06544228,  0.11537051,  0.08104727,
         -0

In [102]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    """예전엔 이게 됐는데 지금은 막힘. 참고용 코드에 이렇게 적혀있으면 바꿔줘야함"""
#     cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
#     cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers
    cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(num_units=5, state_is_tuple=True) for _ in range(3)]
                             , state_is_tuple=True) # 3 layer


    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(1, 5, 5), dtype=float32)
array([[[ 0.00054245, -0.00196731,  0.00115956,  0.00150405,
          0.00154406],
        [ 0.0025574 , -0.00213866, -0.00028167,  0.00331825,
          0.00495022],
        [ 0.00341816, -0.00249767, -0.00038203,  0.00440823,
          0.00559587],
        [ 0.00280092, -0.00471925,  0.002342  ,  0.00467351,
          0.00394952],
        [ 0.00127339, -0.00760254,  0.00449402,  0.00491067,
          0.00398002]]], dtype=float32)


In [8]:
print(x_data.shape)
x_data

(3, 5, 3)


array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size
    pp.pprint(sess.run(_states))

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[ 9.16322917e-02,  8.26223642e-02,  8.59395787e-02,
          1.05789781e-01, -3.39128822e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]],

       [[ 1.12587453e-10,  1.23634580e-09,  5.14535022e-05,
          5.24141453e-03, -8.84465010e-07],
        [ 3.98605593e-11,  3.24333255e-10,  5.35893050e-05,
          2.23060907e-03, -2.74748942e-07],
        [ 1.36351373e-11,  7.89854362e-11,  4.87635298e-05,
          9.38968209e-04, -6.21316119e-08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+

In [114]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)
    outputs_f, outputs_b = outputs

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs_f))
    pp.pprint(sess.run(outputs_b))
    pp.pprint(sess.run(states)) # 마찬가지로 state도 둘로 나눌 수 있다 

array([[[-1.5255492e-02, -3.6959812e-02, -4.8789423e-02,  1.4670210e-02,
          1.6053623e-02],
        [-6.8657249e-02, -2.7031293e-02, -2.0605482e-02, -1.7814202e-02,
         -9.6430913e-02],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]],

       [[-8.0075610e-04, -5.4896959e-06, -1.2623042e-07, -2.0502354e-03,
         -1.8805536e-02],
        [-2.2019839e-04, -6.1834527e-07, -3.1730412e-08, -1.0336440e-03,
         -1.1365689e-02],
        [-5.9916430e-05, -6.9086532e-08, -6.5827295e-09, -4.1806520e-04,
         -5.5703307e-03],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]],

      

In [17]:
# flattern based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

print(x_data.shape)
pp.pprint(x_data)
x_data2 = x_data.reshape(-1, hidden_size) # 이거 안해도 저절로 broadcasting 해줌
print(x_data2.shape)
pp.pprint(x_data2)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data2, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
print(outputs.shape)
pp.pprint(outputs)

(3, 5, 3)
array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)
(15, 3)
array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.],
       [15., 16., 17.],
       [18., 19., 20.],
       [21., 22., 23.],
       [24., 25., 26.],
       [27., 28., 29.],
       [30., 31., 32.],
       [33., 34., 35.],
       [36., 37., 38.],
       [39., 40., 41.],
       [42., 43., 44.]], dtype=float32)
(3, 5, 5)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.],
        [ 205.,  244

In [18]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

# sequence_loss 에서 저절로 [.7] -> [0,1]로 one hot encoding 해줌
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.5967595


In [19]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval(),
      "Loss4: ", sequence_loss4.eval())

Loss1:  0.5130153 Loss2:  0.3711007 Loss3:  1.3132616 Loss4:  0.31326166
